# Programming Assignment #4: Domino Briscas or other popular card games

## Introduction
The study of mathematical models of strategic interactions between rational agents is known as game theory. In addition to logic, systems science, and computer science, it has applications in every area of social science. Economic theory also makes extensive use of game theory concepts [1]. When parties, referred to as players, make interdependent decisions, game theory offers tools for analyzing these situations. Each player must take into account the potential decisions or strategies of the other players as a result of their interdependence [2].

By using game theory, we can formulate games to create agents able to play the game. The purpose of this assignment is to demonstrate that statement by creating an AI agent able to play a game of choice, in this case we chose checkers. We will use algorithms described in Chapter 5 (Adversarial Search) of the class textbook to improve the decisions taken by such agent. 

## Theoretical Background

The Minimax algorithm is a recursive or backtracking algorithm used in game theory and decision-making [3]. It offers the player an ideal move, assuming the opponent is also playing perfectly. To search through the game-tree, the Mini-Max algorithm employs recursion [4]. Because of that, if the game has a large branching factor, this can lead to performance issues.

An improved version of the previous algorithm is the Alpha-beta prunning. Ultimately, the results of both algorithms ought to be the same. Alpha-beta, in contrast to minimax, prunes paths that are certain not to be the best possible states for the current player, that is, max or min. This is their main point of distinction. Therefore, alpha-beta is a better way to implement minimax [5]. Every other child in the tree is eliminated by alpha-beta pruning, leaving only that one child to be investigated. As a result, the tree can now be searched twice as deeply on average than it could before [6].

The Monte Carlo Tree Search (MCTS) heuristic search algorithm is used to find solutions for some decision-making processes, most notably those involved in game play. It combines tree search with random sampling of the search space. Alpha-beta pruning is a search algorithm that aims to reduce the number of nodes in the search tree that are evaluated by the minimax algorithm. The primary distinction between Monte Carlo Tree Search and Alpha-Beta pruning is that MCTS does not require an evaluation function or domain knowledge, whereas Alpha-Beta pruning does.

In some cases, MCTS can outperform Alpha-Beta Pruning, particularly in games with a big search space, hidden information, or stochastic components. This is due to the fact that by utilizing the data gathered from simulations, MCTS is better able to handle these types of scenarios.



## Implementation

Here we implemented a simple version of the game of Checkers. For simplicity, this version of checkers doesn't allow multiple captures in a row, but capture is mandatory.

In [2]:
from aima3.games import Game, RandomPlayer, GameState, AlphaBetaCutoffPlayer, HumanPlayer, MCTSPlayer, AlphaBetaPlayer


class Checkers(Game):
    def __init__(self, initial_state = None):
        super().__init__(self)
        self.board = [[' ' for _ in range(8)] for _ in range(8)]
        self.initialize_board()
        if initial_state is None:
            self.initial = GameState('x',utility=0,moves=self.get_valid_moves('x'), board=self.board)
        else:
            self.initial = initial_state
            self.board = self.initial.board.copy()
    def reset(self):
        self.board = [[' ' for _ in range(8)] for _ in range(8)]
        self.initialize_board()
        self.state =GameState('x',utility=0,moves=self.get_valid_moves('x'), board=self.board)
        return super().reset()

    def initialize_board(self):
        for i in range(8):
            for j in range(8):
                if (i + j) % 2 == 1:
                    if i < 3:
                        self.board[i][j] = 'o'
                    elif i > 4:
                        self.board[i][j] = 'x'
    @classmethod
    def instance(cls):
        return cls
    def display_board(self):
        print('  0 1 2 3 4 5 6 7')
        for i, row in enumerate(self.board):
            print(i, end=' ')
            for cell in row:
                print(cell, end=' ')
            print()

    def move(self, src, dest):
        if self.is_valid_move(src, dest):
            self.board[dest[0]][dest[1]] = self.board[src[0]][src[1]]
            self.board[src[0]][src[1]] = ' '

            # Crown the piece if it reaches the back rank
            if (self.board[dest[0]][dest[1]] == 'x' and dest[0] == 0) or \
               (self.board[dest[0]][dest[1]] == 'o' and dest[0] == 7):
                self.board[dest[0]][dest[1]] = self.board[dest[0]][dest[1]].upper()

            return True
        elif self.is_valid_capture(src, dest, self.board[src[0]][src[1]]):
            self.board[dest[0]][dest[1]] = self.board[src[0]][src[1]]
            self.board[src[0]][src[1]] = ' '

            # Remove the captured piece
            jumped_piece_row = src[0] + (dest[0] - src[0]) // 2
            jumped_piece_col = src[1] + (dest[1] - src[1]) // 2
            self.board[jumped_piece_row][jumped_piece_col] = ' '

            # Crown the piece if it reaches the back rank
            if (self.board[dest[0]][dest[1]] == 'x' and dest[0] == 0) or \
               (self.board[dest[0]][dest[1]] == 'o' and dest[0] == 7):
                self.board[dest[0]][dest[1]] = self.board[dest[0]][dest[1]].upper()

            return True
        else:
            return False
    def is_valid_move(self, src, dest):
        piece = self.board[src[0]][src[1]]
        opponent = 'o' if piece.upper() == 'X' else 'x'
        
        if self.board[dest[0]][dest[1]] != ' ':
            return False

        dy, dx = dest[0] - src[0], dest[1] - src[1]

        if abs(dy) != 1 or abs(dx) != 1:
            return False

        if piece.islower():
            if (piece == 'o' and dy < 0) or (piece == 'x' and dy > 0):
                return False

        return True

    def is_valid_capture(self, src, dest, player):
        if not (0 <= dest[0] < 8 and 0 <= dest[1] < 8):
            return False

        if self.board[dest[0]][dest[1]] != ' ':
            return False

        opponent = 'o' if player.lower() == 'x' else 'x'
        middle_row = src[0] + (dest[0] - src[0]) // 2
        middle_col = src[1] + (dest[1] - src[1]) // 2
        if self.board[middle_row][middle_col].lower() != opponent:
            return False

        dx, dy = dest[0] - src[0], dest[1] - src[1]

        if player.islower():
            if player == 'x' and dx > 0 or player == 'o' and dx < 0:
                return False

        return True

    def is_valid_position(self, row, col):
        return 0 <= row < 8 and 0 <= col < 8

    def get_valid_moves(self, player):
        valid_moves = []
        valid_captures = []
        for row in range(8):
            for col in range(8):
                if self.board[row][col].lower() == player:
                    piece = self.board[row][col]

                    directions = [(1, 1), (1, -1), (-1, 1), (-1, -1)]
                    if piece.islower():
                        directions = [(1, 1), (1, -1)] if piece == 'o' else [(-1, 1), (-1, -1)]

                    for dx, dy in directions:
                        src = (row, col)
                        dest = (row + dx, col + dy)
                        dest_capture = (row + 2 * dx, col + 2 * dy)
                        if 0 <= dest_capture[0] < 8 and 0 <= dest_capture[1] < 8:
                            if self.is_valid_capture(src, dest_capture, piece):
                                valid_captures.append((src, dest_capture))

                        if 0 <= dest[0] < 8 and 0 <= dest[1] < 8:
                            if self.is_valid_move(src, dest):
                                valid_moves.append((src, dest))

        if len(valid_captures) > 0:
            return valid_captures
        return valid_moves
    def actions(self, state):
        return self.get_valid_moves(state.to_move)
    
    def result(self, state, move):
        self.move(move[0], move[1])
        to_move = 'x' if state.to_move == 'o' else 'o'
        return GameState(to_move=to_move, board=self.board, utility=self.utility(None, to_move), moves=self.get_valid_moves(to_move))
    def utility(self, state,player):
        opponent = 'o' if player == 'x' else 'x'
        player_score, opponent_score = 0, 0

        for row in self.board:
            for piece in row:
                if piece.lower() == player:
                    player_score += 1 if piece.islower() else 2
                elif piece.lower() == opponent:
                    opponent_score += 1 if piece.islower() else 2

        return player_score - opponent_score

    def display(self, state):
        return self.display_board()

    def count_pieces(self, piece_type):
        count = 0
        for row in self.board:
            count += row.count(piece_type)
            count += row.count(piece_type.upper())
        return count

    def has_won(self, current_turn):
        opponent = 'o' if current_turn == 'x' else 'x'
        return self.count_pieces(opponent) == 0
    def terminal_test(self, state):
        return self.has_won('x') or self.has_won('o') or len(self.get_valid_moves(state.to_move)) == 0 or (self.count_pieces('x') == 1 and self.count_pieces('o') == 1)
def get_coords(input_str):
    col, row = map(int, input_str.split())
    return row, col

In [3]:
game = Checkers()
player_1 = AlphaBetaCutoffPlayer()
player_2 = MCTSPlayer()

winner = game.play_game(player_1, player_2)

AlphaBetaCutoffPlayer-1 is thinking...
AlphaBetaCutoffPlayer-1 makes action ((5, 2), (4, 3)):
  0 1 2 3 4 5 6 7
0   o   o   o   o 
1 o   o   o   o   
2   o   o   o   o 
3                 
4       x         
5 x       x   x   
6   x   x   x   x 
7 x   x   x   x   
MCTSPlayer-1 is thinking...
MCTSPlayer-1 makes action ((2, 3), (3, 4)):
  0 1 2 3 4 5 6 7
0   o   o   o   o 
1 o   o   o   o   
2   o       o   o 
3         o       
4       x         
5 x       x   x   
6   x   x   x   x 
7 x   x   x   x   
AlphaBetaCutoffPlayer-1 is thinking...
AlphaBetaCutoffPlayer-1 makes action ((5, 6), (4, 7)):
  0 1 2 3 4 5 6 7
0   o   o   o   o 
1 o   o   o   o   
2   o       o   o 
3         o       
4       x       x 
5 x       x       
6   x   x   x   x 
7 x   x   x   x   
MCTSPlayer-1 is thinking...
MCTSPlayer-1 makes action ((3, 4), (5, 2)):
  0 1 2 3 4 5 6 7
0   o   o   o   o 
1 o   o   o   o   
2   o       o   o 
3                 
4               x 
5 x   o   x       
6   x   x   x   x 
7 x   x

['MCTSPlayer-1']

In [4]:
results = game.play_matches(100, players=[player_1, player_2])

  0%|          | 0/100 [00:00<?, ?it/s]

In [5]:
results

{'DRAW': 0, 'AlphaBetaCutoffPlayer-1': 0, 'MCTSPlayer-1': 100}